# Airbnb Data

Investigation of the NYC Airbnb Data

In [40]:
import sys
import textmining
import numpy as np
from __future__ import print_function
from sklearn.feature_extraction.text import TfidfVectorizer
import json
%matplotlib inline
import matplotlib.pyplot as plt
from nltk.tokenize import TreebankWordTokenizer
import copy
from bs4 import UnicodeDammit
#encoding='latin-1'

In [33]:
#Making dictionaries for pre-processing
import csv
reviewsfile = open('jsons/nycreviews.csv', 'rb')
rvreader = csv.reader(reviewsfile)
reviews = {}
for row in rvreader:
    listing_id = row[0]
    comment = row[5]
    if listing_id in reviews.keys():
        reviews[listing_id].append(comment)
    else:
        reviews[listing_id] = [comment]

NameError: name 'io' is not defined

In [18]:
#Dictionary for nyc listings
listingsfile = open('jsons/nyclistings.csv', 'rb')
listingsreader = csv.reader(listingsfile)
listings = {}
for row in listingsreader:
    if row[5] != "neighbourhood":
        listing_id = row[0]
        neighborhood = row[5]
        if neighborhood in listings.keys():
            listings[neighborhood].append(listing_id)
        else:
            listings[neighborhood] = [listing_id]

In [59]:
# listings_reviews -> all reviews for all places within a particular neighborhood
listings_reviews = {}
for neighborhood,listing_ids in listings.iteritems():
    nested_list = [reviews[listing] for listing in listing_ids if listing in reviews.keys()]
    listings_reviews[neighborhood] = [item for sublist in nested_list for item in sublist]

In [37]:
neighborhoods = listings.keys()

In [60]:
#Combining list of reviews for each neighborhood, into a single string
#Also converting all the reviews text into UTF-8 encoding, to build the tf-idf later
def __unicode__(self):
   return unicode(self.some_field) or u''
for neighborhood in listings_reviews:
    print(neighborhood)
    if(len(listings_reviews[neighborhood])>1):
        k=' '.join(listings_reviews[neighborhood])
        m=UnicodeDammit(k)
        s=m.unicode_markup
        listings_reviews[neighborhood]=s.encode("utf-8")
    else:
        print(neighborhood)
        print(len(listings_reviews[neighborhood]))
        if(len(listings_reviews[neighborhood])==1):
            listings_reviews[neighborhood]=listings_reviews[neighborhood][0]
        if(len(listings_reviews[neighborhood])==0):
            listings_reviews[neighborhood]='the' # i think empty text creates an error, hence this
            
scripts=[]
for n in listings_reviews:
    scripts.append(listings_reviews[n])

n_feats = 5000
doc_by_vocab = np.empty([len(neighborhoods), n_feats])

tfidf_vec = TfidfVectorizer(#..
    max_df=0.8, min_df=10, max_features=5000,stop_words='english', norm='l2' )



Ozone Park
Greenpoint
Financial District
Kensington
Mount Eden
Ridgewood
Cambria Heights
Concourse
Port Richmond
Cobble Hill
Inwood
Bay Terrace
Allerton
West Farms
Holliswood
Queens Village
Prospect-Lefferts Gardens
Rockaway Beach
Gramercy
Bensonhurst
North Riverdale
Astoria
Todt Hill
Clinton Hill
Roosevelt Island
Gravesend
University Heights
Port Morris
Stapleton
Westchester Square
Chinatown
Wakefield
Corona
West Village
East Elmhurst
Downtown Brooklyn
Dyker Heights
Upper West Side
Harlem
Bay Ridge
Jamaica Hills
Two Bridges
Jamaica
Co-op City
Huguenot
Huguenot
0
Howard Beach
Highbridge
Brooklyn Heights
Fort Greene
New Dorp Beach
New Dorp Beach
0
NoHo
Briarwood
East Village
Hunts Point
South Beach
Schuylerville
Lower East Side
Edenwald
Baychester
Pelham Bay
Upper East Side
Mariners Harbor
Belmont
Woodlawn
East New York
Richmondtown
Melrose
Grymes Hill
Castle Hill
Bronxdale
Country Club
Country Club
0
Greenwich Village
Forest Hills
Castleton Corners
Arverne
Arden Heights
Westerleigh
Wes

In [75]:
#Making the tfidf matrix using our reviews data
doc_by_vocab=tfidf_vec.fit_transform(scripts)


# Construct a inverted map from feature index to feature value (word) for later use
index_to_vocab = {i:v for i, v in enumerate(tfidf_vec.get_feature_names())}

In [76]:
doc_by_vocab = doc_by_vocab.toarray()

In [86]:
#dictionary that maps neighborhoods to their id in doc_by_vocab matrix
dict_neighborhoods={}
i=0
for neighborhood in neighborhoods:
    dict_neighborhoods[neighborhood]=i
    i+=1

In [87]:
#function to get similarity between query tfidf and neighborhood (name)
def get_sim(doc_tfidf, name2):
    idx2=dict_neighborhoods[name2]
    return np.dot(doc_tfidf,doc_by_vocab[idx2])

In [127]:
#Enter query here, also converting query into a tfidf format
query='clean safe cheap'
query_tfidf = tfidf_vec.transform([query])
query_tfidf=query_tfidf.toarray()


In [128]:
#finding the neighborhood whose reviews match the query the most
maximum=0.0
neighbor=' '
for neighborhood in dict_neighborhoods:
    sim=get_sim(query_tfidf,neighborhood)
    if(sim>maximum):
        maximum=sim
        neighbor=neighborhood
print(neighbor)

Midwood
